In [3]:
import pandas as pd
from pykrx import stock

In [33]:
# 날짜 설정
start_date = '20220101'
end_date = '20241231'

# 모든 ETF 티커 리스트 가져오기
etf_list = stock.get_etf_ticker_list()
len(etf_list), etf_list.describe()


AttributeError: 'list' object has no attribute 'describe'

In [31]:
# ETF 데이터를 가져오는 방법
etf_df = pd.DataFrame()

for etf in etf_list:
    # 시장 인자 없이 기본값을 사용하여 데이터 가져오기
    etf_df[etf] = stock.get_market_ohlcv_by_ticker(etf, start_date, end_date)['종가']

market 옵션이 올바르지 않습니다.


KeyError: '종가'